# Extracción de Precios de Productos - Tiendas Éxito

Este notebook automatiza la extracción y comparación de precios de productos en distintas tiendas de Éxito en Colombia, procesando datos de un archivo Excel y guardando los resultados en un nuevo archivo.

### Importación de librerías

En esta celda se importan las librerías necesarias para el scraping y el manejo de datos. Estas incluyen:
- `time`, `json`, y `base64`: Para manejo de tiempos y codificaciones de cookies.
- `requests`: Para realizar solicitudes HTTP y obtener el contenido de las URLs.
- `pandas`: Para leer y manipular los datos en formato Excel.
- `tqdm`: Para mostrar barras de progreso en la ejecución de bucles.
- `BeautifulSoup` (de `bs4`): Para analizar y extraer datos de HTML.
- `selenium` con `webdriver`: Para automatizar la navegación en Firefox.

In [14]:
import time
import json
import base64
import requests
import xlsxwriter
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

### Funciones auxiliares

Esta celda define varias funciones clave para el scraping:

- `setup_firefox()`: Configura y devuelve un controlador de Firefox usando Selenium.
- `extract_sku(url)`: Extrae el SKU del producto de la página web de Éxito, utilizando BeautifulSoup para analizar el HTML.
- `code_cookies(regionId)`: Codifica las cookies en base64 para la región específica, lo que permite obtener precios según la ubicación.
- `extract_prices(url, regionId)`: Realiza una solicitud API para obtener los precios de un producto dado el SKU y las cookies de la región.

In [46]:
def setup_firefox():
    options = Options()
    driver = webdriver.Firefox(options=options)
    return driver

def extract_sku(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    sku = soup.find('span', class_='product-title_product-title__specification__UTjNc').text.strip().split(':')[-1].strip()
    return sku

def code_cookies(regionId):
    vtex_segment = {"campaigns":None,"channel":"1","priceTables":None,"regionId":regionId,"utm_campaign":None,"utm_source":None,"utmi_campaign":None,"currencyCode":"COP","currencySymbol":"$","countryCode":"COL","cultureInfo":"es-CO","admin_cultureInfo":"es-CO","channelPrivacy":"public"}
    cadena_json = json.dumps(vtex_segment, separators=(',', ':')).encode('utf-8')
    cadena_base64 = base64.b64encode(cadena_json).decode('utf-8')
    return cadena_base64

def extract_prices(url,regionId):
    sku = extract_sku(url)
    vtex_segment = code_cookies(regionId)
    url = "https://www.exito.com/api/product/getProductBySku"
    querystring = {"skuid":str(sku)}
    payload = ""
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://www.exito.com/arroz-diana-500-gr-479512/p",
    "Connection": "keep-alive",
    "Cookie": f"checkout.vtex.com=__ofid=297ea3777be34d8ea4bb2fd73dd5fcd6; checkout.vtex.com=__ofid=297ea3777be34d8ea4bb2fd73dd5fcd6; CheckoutOrderFormOwnership=; _gcl_au=1.1.319435687.1723231502; _ga_W617R65N74=GS1.1.1726842440.21.1.1726842712.59.0.0; _ga=GA1.1.731529654.1723231502; _ga_S44GR46V45=GS1.1.1726842440.20.1.1726842716.55.0.0; __rtbh.lid=%7B%22eventType%22%3A%22lid%22%2C%22id%22%3A%22qvamQVpmQk8pk7BHTsR6%22%7D; _clck=13400qt%7C2%7Cfpc%7C0%7C1682; _tt_enable_cookie=1; _ttp=ddgYMg63156BchtYDn0h6GerStG; _hjSessionUser_1473829=eyJpZCI6IjQ4ZWQxMzlkLWQyOWItNTE2ZS1iODI5LTg3MGZhMDIxYWFlZCIsImNyZWF0ZWQiOjE3MjMyMzE1MDUzNDQsImV4aXN0aW5nIjp0cnVlfQ==; _fbp=fb.1.1723231505471.95295616484517368; exTcIdE=%40%4022%40%4020sr2oAmSiWGXVyDF8d41g%7C1754767509779; VtexRCMacIdv7=20f22b1b-673a-4792-bfcc-4c49cc7e106d; vtex-search-anonymous=bfb5c8d6ce2a4c37b65b98d18d41004b; __gads=ID=f765979ac6db4d60:T=1723231552:RT=1726838101:S=ALNI_MZyFvtdHJruPY5JZAwiT96UKSY2BQ; __gpi=UID=00000ec197bec17a:T=1723231552:RT=1726838101:S=ALNI_MYQVonBO5rIKMX9K_LcNlJGB-4fjQ; __eoi=ID=80c3cb1484dd0bcc:T=1723231552:RT=1726838101:S=AA-Afja-zH6KHGfFXQOeyeIv7brQ; gbi_visitorId=clzn3li8500012a6cgscc1xrp; scarab.visitor=%2276548B39F2F49A1E%22; vtex_segment={vtex_segment}; _hjSession_1473829=eyJpZCI6ImFkZDliY2Y1LWQ3MGYtNDYzZC1iZGM5LTY2ZDNmZjMxMWFkYyIsImMiOjE3MjY4NDI0NDQ1NzcsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; _clsk=1oouc27%7C1726842716035%7C1%7C0%7Cq.clarity.ms%2Fcollect",
}

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    data = json.loads(response.text)
    try:
        regular_price = data[0]['items'][0]['sellers'][0]['commertialOffer']['ListPrice']
    except:
        regular_price = 'No disponible'
    try:
        promotional_price = data[0]['items'][0]['sellers'][0]['commertialOffer']['Price']
    except:
        promotional_price = regular_price
    return promotional_price, regular_price

### Lectura de archivo Excel

Se lee el archivo Excel que contiene las hojas con las URLs de los productos. Cada hoja se guarda como un DataFrame en un diccionario, donde la clave es el nombre de la hoja y el valor es el DataFrame correspondiente.

- `excel_filename`: Ruta al archivo Excel.
- `pd.ExcelFile`: Función de pandas para leer el archivo Excel completo.

In [47]:
excel_filename = '../data/Insuperables_W38.xlsx' 
xls = pd.ExcelFile(excel_filename)
dfs = {sheet: pd.read_excel(xls, sheet_name=sheet) for sheet in xls.sheet_names}

### Definición de regiones y sus códigos

En esta celda se define un diccionario con los códigos de las diferentes regiones de Colombia. Estos códigos serán usados posteriormente en las cookies para obtener los precios específicos según la ubicación.

- `stores`: Diccionario donde las claves son los nombres de las regiones y los valores son los códigos base64 necesarios para las cookies.

In [52]:
stores = {'Bogota':'U1cjZXhpdG9jb2w7ZXhpdG9jb2wwOTM=','Pereira':'U1cjZXhpdG9jb2w7ZXhpdG9jb2wwNjM=', 'Barranquilla':'U1cjZXhpdG9jb2w7ZXhpdG9jb2wwNDE='} 

### Extracción de precios por URL

Aquí se recorren todas las hojas del archivo Excel y se extraen los precios promocionales y regulares de cada URL. Los resultados de cada hoja se almacenan en un DataFrame:

- Se itera sobre las URLs en cada hoja.
- Se utiliza la función `extract_prices` para obtener los precios del producto.
- Si ocurre algún error en la extracción, los precios se marcan como "No disponible".
- Los resultados se guardan en un diccionario donde la clave es el nombre de la hoja.

In [56]:
resultados = {}

for sheet_name, df in dfs.items():
    data_list = []
    for url in tqdm(df['LINK'], desc=f"Procesando URLs de {sheet_name}"):
        try:
            promotional_price, regular_price = extract_prices(url, stores[sheet_name])
            data_list.append({
                'Url': url,
                'Precio Promocional': promotional_price,
                'Precio Regular': regular_price
            })
        except Exception as e:
            #print(f"Error procesando la URL {url}: {e}")
            data_list.append({
                'Url': url,
                'Precio Promocional': 'No disponible',
                'Precio Regular': 'No disponible'
            })
    
    resultados[sheet_name] = pd.DataFrame(data_list)

Procesando URLs de Pereira: 100%|██████████| 90/90 [05:51<00:00,  3.91s/it]


### Guardar resultados en archivo Excel

Los DataFrames generados en la celda anterior se guardan en un nuevo archivo Excel, con cada DataFrame en una hoja diferente. El archivo se guarda en la ruta especificada.

- `pd.ExcelWriter`: Se utiliza para escribir múltiples DataFrames en diferentes hojas de un archivo Excel.
- Se guardan los resultados con los precios extraídos.

In [57]:
output_filename = '../outputs/resultados_precios.xlsx'
with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    for sheet_name, result_df in resultados.items():
        result_df.to_excel(writer, sheet_name=sheet_name, index=False)
print(f"Los resultados se han guardado en el archivo {output_filename}")

Los resultados se han guardado en el archivo ../outputs/resultados_precios.xlsx
